This exercise is prepared for the **Advanced GIS Spatial Big Data** lecture offered in spring 2023 by Moritz Neun (Zürich) in collaboration with the Chair of Geoinformation Engineering (Institute of Cartography and Geoinformation, ETH Zürich).

Date updated: 2023-03-02

Instructor: Dr. Moritz Neun

Materials are prepared by Moritz Neun partially using previous materials by Dr. Yanan Xin, Henry Martin, and Jannik Hamper.

# Discrete Global Grid Systems and Spatial Indices

Spatial indexes are commonly used in geospatial databases and file formats. Commonly R-Trees or Quadtrees are used to index the bounding boxes of every geometry in the dataset (see e.g. [1]). This approach can also be described as "Divide the Objects" Approach [2] which maintains a balanced tree of all objects with their arrangement in space. However this approach requires that the objects are known before a query is made and that the tree (index) is updated whenever a new object (geometry) is added to the dataset.

In contrast, in the “Divide the Space” Approach [2] the total geographic area is subdivided into cells, often arranged hierarchically. In this case, since the total geographic area usually doesn't change, the index doesn't have to be updated when new objects are added. This helps with very large, or permanently updated or streamed data sources. However, the index is usually not as exact and fast for individual object retrieval.

A discrete global grid (DGG) is such a space division or tesselation that covers the entire Earth's surface. Mathematically it is a space partitioning: it consists of a set of non-empty regions that form a partition of the Earth's surface [3]. In a usual grid-modeling strategy, each region or region-point in the grid is called a cell. Common types are (see also [4], [5] and [6]:
* Non-hierarchical grids (UTM Zones,  TIN DEM)
* Hierarchical Grids (ISEA DGGs, Geohash, What3words, PlusCode, S2, H3)
* Equal-area hierarchical grids (DGGRID, OpenEAGGR)

See also these blogposts for further information:
- https://www.uber.com/en-CH/blog/h3/
- https://s2geometry.io/devguide/examples/coverings.html


### S2 vs H3
S2 and H3 are both open source, hierarchical, discrete, and global grid systems (cf. https://h3geo.org/docs/comparisons/s2)

H3:
- aggregation
- visualization
- size consistency

S2:
- indexing
- truyly hierarchical

both:
- joining



### References
* [1] http://postgis.net/workshops/postgis-intro/indexing.html
* [2] https://www.cockroachlabs.com/blog/how-we-built-spatial-indexing/
* [3] Sahr, Kevin; White, Denis; Kimerling, A.J. (2003). "Geodesic discrete global grid systems" https://doi.org/10.1559/152304003100011090doi:10.1559/152304003100011090
* [4] https://en.wikipedia.org/wiki/Discrete_global_grid
* [5] https://agile-giss.copernicus.org/articles/3/41/2022/agile-giss-3-41-2022.pdf
* [6] https://www.ogc.org/projects/groups/dggsswg


Now let's have a deeper look at S2 and H3.

## Imports and Helpers Functions

see also https://s2sphere.readthedocs.io/en/latest/api.html
I you get import errors below (i.e. s2sphere or pyarrow), comment out and run the two following installs.

In [ ]:
# !pip install s2sphere
# !pip install pyarrow

In [ ]:
import folium
import h3
import math
import s2sphere
from IPython.utils.text import columnize

def get_bbox(coords):
    lat_min = 1e7
    lat_max = -1e7
    lon_min = 1e7
    lon_max = -1e7
    for lat, lon in coords:
        lat_min = min(lat_min, lat)
        lat_max = max(lat_max, lat)
        lon_min = min(lon_min, lon)
        lon_max = max(lon_max, lon)
    return [(lat_min, lon_min), (lat_max, lon_max)]

def s2_to_geo_boundary(cell_id):
    cell = s2sphere.Cell(cell_id)
    boundary = []
    for k in range(4):
        ll = s2sphere.LatLng.from_point(cell.get_vertex(k))
        boundary.append((ll.lat().degrees, ll.lng().degrees))
    return boundary

def plot_s2(cell_id):
    print(cell_id)
    return plot_geometries([s2_to_geo_boundary(cell_id)])

def plot_s2cells(cell_ids):
    polygons = []
    for cell_id in cell_ids:
        print(cell_id)
        polygons.append(s2_to_geo_boundary(cell_id))
    return plot_geometries(polygons)
    
def plot_h3(h):
    return plot_geometries([h3.h3_to_geo_boundary(h=h, geo_json=False)])

def plot_h3s(h3s):
    polygons = []
    for h in h3s:
        print(h)
        polygons.append(h3.h3_to_geo_boundary(h=h, geo_json=False))
    return plot_geometries(polygons)

def plot_geometries(polygons=[], points=[]):
    f = folium.Figure(width=600, height=300)
    m = folium.Map(
        tiles='https://tiles.stadiamaps.com/tiles/osm_bright/{z}/{x}/{y}{r}.png',
        attr='(C) Stadia Maps, (C) OpenMapTiles (C) OpenStreetMap contributors',
        zoom_start=20, max_zoom=20).add_to(f)
    for polygon in polygons:
        folium.Polygon(polygon, color='#ff0000', opacity=1).add_to(m)
    for point in points:
        folium.Marker(point, color='#0000ff', opacity=1).add_to(m)
    bb = get_bbox([c for p in polygons for c in p])
    m.fit_bounds(bb)
    return m

## Overview S2 and H3

In [ ]:
print('** S2 Functions **\n')
print(columnize(dir(s2sphere), displaywidth=100))
print('\n** H3 Functions **\n')
print(columnize(dir(h3), displaywidth=100))

## S2 Examples
You can use the `s2sphere` library to get the S2 cell for any coordinates.

In [ ]:
cell = s2sphere.Cell.from_lat_lng(
    s2sphere.LatLng.from_degrees(47.40821091064011, 8.50744520145277))
print(cell)
plot_s2(cell.id().parent(21))

You can your the `parent` method to traverse the hierarchy of the index. Try to set a different hierarchy level in the `parent` method

In [ ]:
plot_s2(cell.id().parent(15))

The S2 hierarchy is a perfect subdivision into 4 children.

In [ ]:
plot_s2cells(cell.id().parent(15).children())

## H3 Examples

You can use the h3 library to get the H3 cell for any coordinates. Here the hierarchy level can be set explicitly.

In [ ]:
h = h3.geo_to_h3(47.40821091064011, 8.50744520145277, 15)
print(h)
plot_h3(h)

In [ ]:
h = h3.geo_to_h3(47.40821091064011, 8.50744520145277, 9)
print(h)
plot_h3(h)

The H3 hierarchy is a subdivision into 7 children.


In [ ]:
h3c = list(h3.h3_to_children(h, 10))
plot_h3s(h3c)

The H3 hierarchy is **not** a perfect subdivision and only matches roughly.
So it's not advisable to use combinations of cells at different levels.
Note: `plot_h3s` takes a list of H3 cell ids and plots them on an interactive map. 

In [ ]:
h3c.append(h)
plot_h3s(h3c)

## Comparsion of S2 and H3
https://h3geo.org/docs/comparisons/s2
discrete hierarchical spatial index
not fully like OGC Discrete Global Grid System (DGGS), https://www.ogc.org/projects/groups/dggsswg ... but close.

Typical use-cases and properties

H3:
- aggregation
- visualization
- better size consistency

S2:
- indexing
- truyly hierarchical

both:
- joining

## Indexing
We now compare the indices of H3 and S2. 

### H3 Index
The following shows the indices of the different hierarchy levels of **H3** as hex and base10 

In [ ]:
for l in range(2, 16):
    token = h3.geo_to_h3(47.40821091064011, 8.50744520145277, l)
    print(f'{token}\t\t\t{int(token, 16):20}')

### S2 Index
The following shows the indices of the different hierarchy levels of **S2** as hex and base10 

In [ ]:
cell = s2sphere.Cell.from_lat_lng(s2sphere.LatLng.from_degrees(47.40821091064011, 8.50744520145277))
for l in range(2, 30):
    token = cell.id().parent(l).to_token()
    print(f'{token:<16}\t\t{int(token, 16):20}')

For building a large-scale, distributed map search and reverse-geocoding system the S2 tokens can be stored as text. This means that we can use text search engines location search. One of the great advantages of searching locations as texts for companies is that you only need a single infrastructure to search for text and locations. E.g., Google Search and Google Maps can use the same infrastructure!

## Trie as Prefix Search Engine
A [trie](https://en.wikipedia.org/wiki/Trie), also called digital tree or prefix tree, is a type of k-ary search tree, a tree data structure used for locating specific keys from within a set. We now use a simple Python Trie implementation to build a basic text search engine on S2 indices.

In [ ]:
class TrieNode:
 
    def __init__(self, char, data=None):
        self.char = char
        self.data = data
        self.is_end = False
        self.children = {}


class Trie:
 
    def __init__(self):
        self.root = TrieNode("")
     
    def insert(self, word, data=None):
        node = self.root
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
 
                new_node = TrieNode(char, data)
                node.children[char] = new_node
                node = new_node
        node.is_end = True
         
    def dfs(self, node, pre):
        if node.is_end:
            self.output.append(((pre + node.char), node.data))
        for child in node.children.values():
            self.dfs(child, pre + node.char)
         
    def search(self, x):
        node = self.root
        for char in x:
            if char in node.children:
                node = node.children[char]
            else:
                return []
        self.output = []
        self.dfs(node, x[:-1])
        return self.output

In [ ]:
tr = Trie()
tr.insert("here")
tr.insert("hear")
tr.insert("he")
tr.insert("hello")
tr.insert("how ")
tr.insert("her")

In [ ]:
%%time
tr.search("he")

### Prepare data

Initialize 1 M random points in greater Zurich for nearby search.

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
import numpy as np
import pandas as pd
import shapely

def create_random_points(overwrite=False):
    miny, maxy, minx, maxx = (47.08300, 47.57800, 8.34500, 8.78100)
    number = 1000000
    x = np.random.uniform(minx, maxx, number)
    y = np.random.uniform(miny, maxy, number)
    df = pd.DataFrame({'x':x, 'y':y})
    gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.x, df.y))
    gdf = gdf.set_crs('EPSG:4326')
    gdf.reset_index(inplace=True)
    gdf = gdf.rename(columns={'index': 'uid'})
    if overwrite:
        tgdf.to_parquet("zurich_random_points.parquet")
    return gdf

# Load the already generated random points file.
# Alternatively call create_random_points to create new ones.
gdf = geopandas.read_parquet('zurich_random_points.parquet') # precomputed for stability and speed
gdf

We now generate a cirle as search area with the goal ot later search for all points within this circle

In [ ]:
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
from pyproj import Transformer

project_to_meters = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True).transform
project_to_latlng = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True).transform

def buffer_in_meters(lng, lat, radius, quad_segs=3):
    pt_latlng = Point(lng, lat)
    pt_meters = transform(project_to_meters, pt_latlng)
    buffer_meters = pt_meters.buffer(radius, quad_segs=quad_segs)
    buffer_latlng = transform(project_to_latlng, buffer_meters)
    return buffer_latlng

buffer_in_meters(8.73, 47.2, 100)

### Initialize the search circle and visualize it on a map

In [ ]:
# TODO initialize a GeoDataFrame containing a circle of approx 100m around the search point.
search_circle = ??
gdfc = geopandas.GeoDataFrame(geopandas.GeoSeries(search_circle), columns=['geometry'])
gdfc = gdfc.set_crs('EPSG:4326')
plot_geometries(polygons=[[(y,x) for x,y in search_circle.exterior.coords]])

### Search for all points within the circle using a for loop

In [ ]:
%%time
# TODO do an simple for loop over `gdf` to find the rows that intersect the `search_circle` geometry.
for ??


### Search for all points within the circle using Pandas

Use the pandas overlay function to identify all points within the search circle.
hint: Here is the relevant documentation [https://geopandas.org/en/stable/docs/user_guide/set_operations.html](https://geopandas.org/en/stable/docs/user_guide/set_operations.html)

In [ ]:
%%time
# TODO do an intersection search using the gdfc polygon on the points in gdf.
gdfi = ??
gdfi

### Investigating the Pandas spatial index
Pandas uses the PyGEOSSTRTreeIndex. We can see that by calling `.sindex` on the geometry column. 
Note that by calling `.sindex` we are initializing the index i.e. building the RTree which makes querying it later faster.

In [ ]:
%%time
gdf['geometry'].sindex

With the index already built, the identification of the indices of the geometries is extremly fast!

In [ ]:
%%time
gdf['geometry'].sindex.query(search_circle, predicate='intersects')

In [ ]:
%%timeit
gdf['geometry'].sindex.query(search_circle, predicate='intersects')

There is still some overhead for retrieving the rows based on the index

In [ ]:
%%time
gdf.loc[gdf['geometry'].sindex.query(search_circle, predicate='intersects')]

#### Plot the results

In [ ]:
plot_geometries(polygons=[[(y,x) for x,y in search_circle.exterior.coords]],
                points=[ll for ll in zip(gdfi['y'], gdfi['x'])])

### Evaluation of S2 for spatial indexing

#### Built S2 index in the data
Add the S2 cells as fixed grid index. This step is the first step of building a (spatial) S2 index. Your implementation is in Python and will therefore be a bit slow. In practice this could be speed up further if necessary. 

In [ ]:
# TODO compute a new column "s2cell" containing the hex token of the S2 cell ID for every point (x,y).
gdf['s2cell'] = ??
gdf

#### Identify S2 index of the query
Find the S2 cell that corresponds to the search circle. For now, we take the S2 cell of level 16 (about 200 meters in size) where the circle center lies in.

In [ ]:
s2search = s2sphere.Cell.from_lat_lng(
    s2sphere.LatLng.from_degrees(47.2, 8.73)).id().parent(16)
s2token = s2search.to_token()
print(s2token)
plot_geometries(polygons=[s2_to_geo_boundary(s2search)])

#### Prefix search using Pandas startswith
Note: 
- We have stored the S2 cell id where the circle is located in, in the variable `s2search` 
- Pandas `startswith` identifies all strings that start with the string "s2token" in a for loop like way

In [ ]:
%%time
print(s2search)
gdfs = gdf[gdf['s2cell'].str.startswith(s2token)]
gdfs

In [ ]:
plot_geometries(polygons=[s2_to_geo_boundary(s2search)], points=[ll for ll in zip(gdfs['y'], gdfs['x'])])

### Prefix search using our custom Trie
##### Build the Trie

In [ ]:
%%time
gtr = Trie()
for c, i, x, y in zip(gdf['s2cell'], gdf['uid'], gdf['x'], gdf['y']):
    gtr.insert(c, (i, x, y))

In [ ]:
s2token

#### Search using Trie

In [ ]:
%%time
gtr.search(s2token)

In [ ]:
%%timeit
gtr.search(s2token)

#### S2 Coverings for approximating geometries

In the example with a circle, a single S2 cell usually is not a very good approximation of the geometry.
In that case the S2RegionCoverer is able to approximate any given geometry with a set of S2 cells.

For more information https://s2geometry.io/devguide/examples/coverings.html

In [ ]:
from s2sphere import RegionCoverer, Cap, LatLng, Angle

coverer = RegionCoverer()
coverer.max_cells = 50
coverer.min_level = 17
coverer.max_level = 18

lat, lng = (47.2, 8.73)
radius_m = 150
latlng = LatLng.from_degrees(float(lat), float(lng)).normalized().to_point()
earth_radius_m = 6.371 * 1e12
region = Cap(latlng, radius_m / earth_radius_m)
print(region)

covering = coverer.get_covering(region)

plot_s2cells(covering)

### Include circle in plot

S2 tokens and their children/parent token can have the same length and only a difference in the last character. Hence, in order to make the S2 search tokens prefix searchable, we use the children tokens and omit the last character. This gives us all possible token-prefix combinations that are valid. If you are interested, you can find further information at https://s2geometry.io/devguide/s2cell_hierarchy

In [ ]:
def covering_to_prefixes(covering):
    tokens = set()
    for c in covering:
        for ch in c.children():
            tokens.add(ch.to_token()[:-1])
    return tokens
        
covering_prefixes = covering_to_prefixes(covering)
print(covering_prefixes)

We now have to search our dataset for every prefix in the covering prefixes

In [ ]:
%%time
#TODO implement a simple search in the Trie (variable `gtr`) to find all points
#that are matching the `covering_prefixes`
results = set()
for ??
print(results)

So with this approximation we're receiving roughly the same results as with the previous pandas circle search.
The speed equals roughly the pure index retrieval of the PGEOS R-Tree index used in Pandas without the additional overhead for retrieving the data.
For exact results we would need to perform an intersect between the search circle and the candidate points.

In [ ]:
plot_geometries(polygons=[s2_to_geo_boundary(c) for c in covering],
                points=[(i[2],i[1]) for c, i in results])

Here are the pandas results as a reference

In [ ]:
plot_geometries(polygons=[[(y,x) for x,y in search_circle.exterior.coords]],
                points=[ll for ll in zip(gdfi['y'], gdfi['x'])])